unity catelog data governanace
3-Level Namespace: Catalog → Schema (Database) → Table


In [0]:
# create catelog and schema governance setup 3-level namespace
catalog_name ="ecom_catalog"
schema_name = "ecom_schema"

try:
    spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
    spark.sql(f"USE CATALOG {catalog_name}")
    print(f"catalog created : {catalog_name}")

    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
    spark.sql(f"USE SCHEMA {schema_name}")
    print (f"using namespace : {catalog_name}.{schema_name}")
except Exception as e:
    print("cannot create catalog or schema")
    print(e)

    spark.sql("USE CATALOG hive_metastore")
    spark.sql("CREATE SCHEMA IF NOT EXISTS {schema_name}")
    spark.sql("USE SCHEMA {schema_name}")



catalog created : ecom_catalog
using namespace : ecom_catalog.ecom_schema


In [0]:
%sql

SELECT current_catalog(), current_schema();

current_catalog(),current_schema()
ecom_catalog,ecom_schema


In [0]:
#  register managed tables
base_path = "/Volumes/workspace/ecommerce/ecommerce_data"

paths = {
    "bronze": f"{base_path}/bronze/events",
    "silver": f"{base_path}/silver/events",
    "gold": f"{base_path}/gold/events"
}

spark.sql("USE CATALOG ecom_catalog")
spark.sql("USE SCHEMA ecom_schema")

def create_managed_table(table_name, source_path):
    print(f"📁 Registering {table_name} to Managed Table...")

    df = spark.read.format("delta").load(source_path)

    #full_table_name = f"ecom_catalog.ecom_schema.{table_name}"
    df.write.format("delta").mode("overwrite").saveAsTable(table_name)

    print(f"   ✅ {table_name} registered successfully.")

# create/register managed tables for all layers
create_managed_table("bronze_events", paths["bronze"])
create_managed_table("silver_events", paths["silver"])
create_managed_table("gold_events", paths["gold"])

📁 Registering bronze_events to Managed Table...
   ✅ bronze_events registered successfully.
📁 Registering silver_events to Managed Table...
   ✅ silver_events registered successfully.
📁 Registering gold_events to Managed Table...
   ✅ gold_events registered successfully.


In [0]:
%sql
SHOW TABLES IN ecom_catalog.ecom_schema;

database,tableName,isTemporary
ecom_schema,bronze_events,false
ecom_schema,gold_events,false
ecom_schema,silver_events,false


In [0]:
%sql
SELECT current_user();

current_user()
guna.thangavel36@gmail.com


In [0]:
# set permissions
print("Applying Access Policies...")

spark.sql("GRANT SELECT ON TABLE gold_events TO `user_id`") # replaced `user_id` with `user_email_id` to mask user email
spark.sql("REVOKE SELECT ON TABLE silver_events FROM `user_id`")

print(" Permissions applied.")


Applying Access Policies...
 Permissions applied.


In [0]:
%sql
select * from gold_events limit 5

category_id,category_code,views,carts,purchases,revenue,cart_to_purchase_rate
2088750570935419494,construction.tools.painting,28,0,0,null,0.0
2106075725441269865,null,16077,291,181,7139.08,62.19931271477663
2070005009256284935,null,16540,132,40,5659.319999999999,30.303030303030305
2076715358323998979,null,49,0,1,65.33,0.0
2053013565782098913,apparel.shoes,91873,2893,1896,254128.66999999995,65.53750432077429


In [0]:
%sql
SELECT * FROM silver_events LIMIT 5;

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,price_tier
2019-10-01T02:19:09.000Z,view,1002527,2053013555631882655,electronics.smartphone,apple,771.96,518297945,3028914b-cc78-f2c8-9a27-0a1f959f7b2c,2019-10-01,premium
2019-10-01T02:20:29.000Z,view,1005017,2053013555631882655,electronics.smartphone,huawei,95.11,518908920,0b131d55-5abd-48c1-99a2-68e503feaf3f,2019-10-01,premium
2019-10-01T02:23:56.000Z,view,5701108,2053013553970938175,auto.accessories.player,pioneer,120.98,512949728,383d04a3-754a-4a54-ad66-bdf212de779c,2019-10-01,premium
2019-10-01T02:25:09.000Z,view,4300211,2053013552385491165,null,vitek,23.14,512848363,e1eea5bb-1012-4590-840e-b0bc445b3ba8,2019-10-01,mid
2019-10-01T02:27:13.000Z,view,28715116,2053013565480109009,apparel.shoes.keds,escan,51.22,539238830,25048d1c-3878-4b30-af11-52690dd2dc3f,2019-10-01,premium


In [0]:
# SECURE VIEW WITH MASKING

from pyspark.sql.functions import sha2, col

view_name = "gold_product_performance"

spark.sql(f"""
    CREATE OR REPLACE VIEW {schema_name}.{view_name} AS
    SELECT
        event_time,
        event_type,
        product_id,
        price,
        brand,
        sha2(CAST(user_id AS STRING), 256) AS hashed_user_id
    FROM silver_events
""")

print(f"View Created: {schema_name}.{view_name}")
display(spark.read.table(f"{schema_name}.{view_name}").limit(3))

View Created: ecom_schema.gold_product_performance


event_time,event_type,product_id,price,brand,hashed_user_id
2019-10-01T02:19:09.000Z,view,1002527,771.96,apple,c85e85b6df1c4af30528527b1d086a7405ba5c3b70c010c973475ec9e9c454f5
2019-10-01T02:20:29.000Z,view,1005017,95.11,huawei,2b174b9c6c17c72bb3e1d213041e92d22d640904da716fd799895028a9d96dc4
2019-10-01T02:23:56.000Z,view,5701108,120.98,pioneer,cd215088ee2e9e5b3de403920f52a9152c774c61f3faa51dcbb3475288f60397


In [0]:
print("📊 Preview of Masked Marketing View:")
display(spark.read.table(f"{schema_name}.{view_name}").limit(5))